<img src="https://m1.quebecormedia.com/emp/emp/bender8175858f-7dbd-4927-85c5-85557e800b98_ORIGINAL.jpg?impolicy=crop-resize&x=0&y=0&w=1000&h=745&width=925&height=925" style="float: left; margin: 30px; height: 75px">

# Predicting Customer Satisfaction on Rent the Runway

## III. Modeling
### Katrin Ayrapetov


<font style="font-size: 2rem; color: blue">


 
</font>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import joblib
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [2]:
import itertools
from sklearn import model_selection
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

In [3]:
df =  pd.read_csv('../Data/df_clean.csv')

In [4]:
#Binarize the Predictor Variable: Satisfaction Rating 
# 1: Not Satisfied with Rental (Satisfaction Rating of 1,2,3) 
# 0: Satisfied with Rental (Satisfaction Rating 4,5)
df['Rating'] = np.where(df['Rating'] <= 3, 1, 0)

In [5]:
#We are dealing with an unbalanced class. 
df["Rating"].value_counts(normalize=True)

0    0.833589
1    0.166411
Name: Rating, dtype: float64

In [6]:
#df.drop[columns=[]]

In [7]:
#Create a target variable vector. 
y = df.Rating.values

In [8]:
#initiate the kfold class from model_selection module
#The folds are made by preserving the percentage of samples for each class.
kf = model_selection.StratifiedKFold(n_splits=5)

In [9]:
#Create a column to hold kfold labels 
df["kfold"] = -1

In [10]:
#Reshuffle the rows. 
df = df.sample(frac=1).reset_index(drop=True)

In [11]:
for f, (t_,v_) in enumerate(kf.split(X=df,y=y)):
    df.loc[v_,"kfold"] = f

In [12]:
#Check that the same number of observations is in each fold. 
df.kfold.value_counts()

0    32266
1    32266
2    32266
3    32266
4    32265
Name: kfold, dtype: int64

In [13]:
#Check that the target variable has the same distribution  the same in each fold. 
for k in range(5):
    print(f"fold: k = {k}")
    print(df[df.kfold==k].Rating.value_counts(normalize=True))
    

fold: k = 0
0    0.835276
1    0.164724
Name: Rating, dtype: float64
fold: k = 1
0    0.831711
1    0.168289
Name: Rating, dtype: float64
fold: k = 2
0    0.835276
1    0.164724
Name: Rating, dtype: float64
fold: k = 3
0    0.83354
1    0.16646
Name: Rating, dtype: float64
fold: k = 4
0    0.83214
1    0.16786
Name: Rating, dtype: float64


In [14]:
#Drop the unnecessary columns 
df.drop(columns=["Brand","Dress_Description","Retail_price","Details"],inplace=True)
df.dropna(inplace=True)

In [15]:
df_train = df[df.kfold != 3].reset_index(drop=True)
df_valid = df[df.kfold == 3].reset_index(drop=True)

In [16]:
df_train.drop(columns=["kfold"],inplace=True)
df_valid.drop(columns=["kfold"],inplace=True)

In [17]:
#This is the train / test data sets for data as it is. 
df_train.to_csv('../Data/df_train_fashion.csv', header=True, index=False)
df_valid.to_csv('../Data/df_valid_fashion.csv', header=True, index=False)

In [19]:
#Try an oversampling technique on the training data set. 
#This would ensure that the minority class was oversampled to have half the number of examples as the majority class.
# define oversampling strategy
oversample_50 = RandomOverSampler(sampling_strategy=0.5)

In [20]:
features = ['Label', 'Size', 'Overall_fit', 'Rented_for', 'Size_usually_worn',
            'Height', 'Age', 'Bust_size', 'Body_type', 'Weight',  'Date',
            'Rent_price', 'Number_of_reviews', 'Sleeves', 'Neckline', 'Dress_Style','BMI']
target = ['Rating']

In [22]:
# Split into test and train
X_train = df_train[features]
y_train = df_train[target].to_numpy()

In [26]:
df_train = oversample_50.fit_resample(df_train,df_train.Rating)

In [27]:
X_over, y_over = oversample_50.fit_resample(X_train, y_train)

In [33]:
y_over = pd.DataFrame(y_over)
df_train_oversampled = pd.concat([X_over, y_over], axis=1)
df_train_oversampled.rename(columns={0:'Rating'}, inplace=True)

In [44]:
df_train_oversampled.columns

Index([            'Label',              'Size',       'Overall_fit',
              'Rented_for', 'Size_usually_worn',            'Height',
                     'Age',         'Bust_size',         'Body_type',
                  'Weight',              'Date',        'Rent_price',
       'Number_of_reviews',           'Sleeves',          'Neckline',
             'Dress_Style',               'BMI',                   0],
      dtype='object')

In [49]:
df_train_oversampled.to_csv('../Data/df_train_fashion_oversampled.csv', header=True, index=False)